In [0]:
%run ./Designer

In [0]:
class Ingestion():
    def __init__(
        self
        ,FeedTopics_file: str
    ):
        self.FeedTopics = read_yaml(FeedTopics_file)
        self.dict_Feed = self.get_Feed()
        self.topics = pd.DataFrame(columns=['topic_name','topic_schema','topic_BronzeTblName'])
        self.list_of_topics = ['books','customers','orders']

    def get_Feed(self)-> dict:
        FeedTopics = self.FeedTopics
        return {
            "feed_path": FeedTopics['feed']['path']
            ,"feed_schema": FeedTopics['feed']['schema']
        }

    def get_Topics(self):
        FeedTopics = self.FeedTopics
        topics = FeedTopics['topics']
        for topic in topics:
            topic_name = topic['name']
            topic_schema = topic['schema']
            topic_BronzeTblName = topic['BronzeTblName']
            self.topics = appendTo_DF(self.topics,{'topic_name':topic_name,'topic_schema':topic_schema,'topic_BronzeTblName':topic_BronzeTblName})

    def GenerateDLTScript(self):
        feed_path = self.dict_Feed['feed_path']
        topics = 
        generate_dlt_script(feed_path,)
